In [ ]:
import re

import pandas as pd
from pathlib import Path
from ploting import *

In [ ]:
root_path = "guyot"

In [ ]:
time_re = re.compile("apply - inverse mass -.*\|\s+(\d+)\s+\|\s+(\d+(\.\d+)?)")

In [ ]:
iter_re = re.compile("DEAL:.*::iterations:\s*(\d+)")

In [ ]:
batch_iter_re = re.compile("\[(\d+)\|(\d+)\|")

In [ ]:
def config_data(filename: str):
    items = filename.replace(".out", "").split("-")
    return dict((key, value) for key, value in [item.split("_", maxsplit=1) for item in items])

In [ ]:
data = []
for file in Path(f"{root_path}/data").iterdir():
    with open(file, "r") as input:
        if t_m := time_re.search(input.read()):
            data.append({**config_data(file.name),
                         "time": t_m.group(2), "timesteps": t_m.group(1)})
df = pd.DataFrame(data).astype({"dim": int, "ncells": int, "degree": int, "timesteps": int, "time": float}).drop(columns=["cpu", "dryrun", "fmt", "refinement", "dim", "batchsize", "exec", "gpu"])
df

In [ ]:
def get_iters(testcase, refinement, degree):
    data = dict()
    for file in Path(f"{root_path}/data").iterdir():
        if f"testcase_{testcase}" in file.name and f"refinement_{refinement}" in file.name and f"degree_{degree}"in file.name:
            matrix_iters = list()
            batch_iters = list()
            timesteps = 1
            with open(file, "r") as input:
                for i_m in iter_re.finditer(input.read()):
                    matrix_iters.append(int(i_m.group(1)))
            with open(file, "r") as input:
                for l in input:
                    if i_m:= batch_iter_re.search(l):
                        batch_iters += map(int, l[i_m.end():-2].split(","))
            with open(file, "r") as input:
                if t_m := time_re.search(input.read()):
                    timesteps = int(t_m.group(1))
            if matrix_iters:
                data["matrix"] = pd.Series(matrix_iters)
            if batch_iters:
                data["batch"] = pd.Series(batch_iters)
    return pd.DataFrame(data), timesteps

In [ ]:
iters, timesteps = get_iters(1, 4, 5)
print(iters.matrix.mean())
print(iters.batch.describe())

In [ ]:
iters_nu, ts_nu = iters, timesteps
iters_u, ts_u = get_iters(0, 5, 5)
bins = range(int(min(iters_u.batch.min(), iters_nu.batch.min())), iters_nu.batch.max() + 1)
weights = [[1 / ts_u] * len(iters_u.batch), [1 / ts_nu] * len(iters_nu.batch)]
fig, ax = plt.subplots(dpi=150, layout="tight")
ax.hist([iters_u.batch, iters_nu.batch], bins=bins, weights=weights, align="left", histtype="stepfilled", alpha=0.9)
ax.axvline(iters_u.matrix.mean(), c='C0', ls=":")
ax.axvline(iters_nu.matrix.mean(), c='C1', ls=":")
ax.set_title("Distribution of Iterations until Convergence")
ax.legend(["Non Uniform", "Uniform"])
ax.set_ylabel("Frequency")
ax.set_xlabel("Number of Iterations")
ax.grid(False)

In [ ]:
fig, ax = plt.subplots(dpi=150, layout="tight")
iters.batch.hist(axes=ax, bins=range(iters.batch.min(), iters.batch.max() + 1), weights=[1 / timesteps] * len(iters.batch),
                 align="left")
ax.axvline(iters.matrix.mean(), c='r')
ax.set_title("Iteration Distribution (Non-Uniform)")
ax.legend(["Full matrix"])
ax.grid(False)

In [ ]:
iters, timesteps = get_iters(0, 5, 5)
fig, ax = plt.subplots(dpi=150, layout="tight")
iters.batch.hist(axes=ax, bins=range(iters.batch.min(), iters.batch.max() + 1), weights=[1 / timesteps] * len(iters.batch),
                 align="left")
ax.axvline(iters.matrix.mean(), c='r')
ax.set_title("Iteration Distribution (Uniform)")
ax.legend(["Full matrix"])
ax.grid(False)

In [ ]:
df['time_normalized'] = df.time / df.timesteps / df.ncells
df

In [ ]:
df["dofs/s"] = (df.ncells * (4 * (df.degree + 1) ** 2)) / df.time / df.timesteps
df

In [ ]:
df0 = df[df.testcase == "0"]
df1 = df[df.testcase == "1"]

In [ ]:
t1 = df1.set_index(["ncells", "degree", "solver"])["dofs/s"].sort_index()
t1.unstack([1, 2])[5].plot(logx=True, logy=True)

In [ ]:
t1 = df1.set_index(["ncells", "degree", "solver"])["time_normalized"].sort_index()
t0 = df0.set_index(["ncells", "degree", "solver"])["time_normalized"].sort_index()

In [ ]:
t0.unstack([1, 2])[5]

In [ ]:
t1.unstack([1, 2])[5]

In [ ]:
fig = plt.figure(layout="tight", dpi=200)
gs = fig.gs = fig.add_gridspec(1, 3, wspace=0)
axes = gs.subplots(sharex=True, sharey=True)
data = t1.unstack([1, 2])
data[3].plot(ax=axes[0], logx=True, logy=True, legend=False)
axes[0].set_title("P3")
data[5].plot(ax=axes[1], logx=True, logy=True, legend=False)
axes[1].set_title("P5")
data[7].plot(ax=axes[2], logx=True, logy=True, legend=False)
axes[2].set_title("P7")
axes[0].legend(["Batches", "Full Matrix"])
axes[0].set_ylabel("Time / (Time Step / Cells)")
axes[0].set_xlabel("Cells")
axes[1].set_xlabel("Cells")
axes[2].set_xlabel("Cells")
fig.suptitle("Non Uniform Mesh")

In [ ]:
fig = plt.figure(layout="tight", dpi=200)
gs = fig.gs = fig.add_gridspec(1, 3, wspace=0)
axes = gs.subplots(sharex=True, sharey=True)
data = t0.unstack([1, 2])
data[3].plot(ax=axes[0], logx=True, logy=True)
axes[0].set_title("P3")
data[5].plot(ax=axes[1], logx=True, logy=True, legend=False)
axes[1].set_title("P5")
data[7].plot(ax=axes[2], logx=True, logy=True, legend=False)
axes[2].set_title("P7")
axes[0].legend(["Batches", "Full Matrix"])
axes[0].set_ylabel("Time / (Time Step / Cells)")
axes[0].set_xlabel("Cells")
axes[1].set_xlabel("Cells")
axes[2].set_xlabel("Cells")
fig.suptitle("Uniform Mesh")

In [ ]:
a = pd.Series([1,2,3])
b = pd.Series([4,5,6, 7])
p = pd.DataFrame({"a": a, "b": b})
p

In [ ]:
p.columns